# MongoDB

MongoDB is a document database. It stores JSON objects.

- [Documentation](https://docs.mongodb.com)
- [Query selectors](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors)

In [1]:
from pymongo import MongoClient, GEOSPHERE
from bson.objectid import ObjectId
from bson.son import SON

In [2]:
import requests
from bson import json_util

In [3]:
import collections
from pathlib import Path

In [4]:
from pprint import pprint

## Set up

This connects to the MongoDB daemon

In [5]:
client = MongoClient()

This specifies the database. It does not matter if it does not exist.

In [6]:
client.drop_database('starwars')

In [7]:
db = client.starwars

This specifies a `collection`

In [8]:
people = db.people

Check what collections are in the database. Note that the `people` collection is only created when the first value is inserted.

In [9]:
db.list_collection_names()

[]

## Get Data

In [10]:
base_url = 'https://swapi.co/api'

In [11]:
resp = requests.get('https://swapi.co/api/people/1')
data = resp.json()

In [12]:
data

{'birth_year': '19BBY',
 'created': '2014-12-09T13:50:51.644000Z',
 'edited': '2014-12-20T21:17:56.891000Z',
 'eye_color': 'blue',
 'films': ['https://swapi.co/api/films/2/',
  'https://swapi.co/api/films/6/',
  'https://swapi.co/api/films/3/',
  'https://swapi.co/api/films/1/',
  'https://swapi.co/api/films/7/'],
 'gender': 'male',
 'hair_color': 'blond',
 'height': '172',
 'homeworld': 'https://swapi.co/api/planets/1/',
 'mass': '77',
 'name': 'Luke Skywalker',
 'skin_color': 'fair',
 'species': ['https://swapi.co/api/species/1/'],
 'starships': ['https://swapi.co/api/starships/12/',
  'https://swapi.co/api/starships/22/'],
 'url': 'https://swapi.co/api/people/1/',
 'vehicles': ['https://swapi.co/api/vehicles/14/',
  'https://swapi.co/api/vehicles/30/']}

In [13]:
def convert(data):
    """Nest inner API calls one level dowwn."""
    for key, val in data.items():
        if isinstance(val, list):
            data[key] = [requests.get(value).json() for value in val 
                         if isinstance(value, str) and value.startswith(base_url)]
        else:
            if isinstance(val, str) and val.startswith(base_url):
                data[key] = requests.get(val).json() 
    return data

In [14]:
luke = convert(data)

In [15]:
def convert_str(s):
    if '.' in s:
        try:
            s = float(s)
        except:
            pass
    else:
        try:
            s = int(s)
        except:
            pass
    return s

def to_numeric(data):
    for key in data:
        val = data[key]
        if isinstance(val, str):
            data[key] = convert_str(val)
        elif isinstance(val, dict):
            for k, v in val.items():
                if isinstance(v, str):
                    val[k] = convert_str(v)
        elif isinstance(val, list):
            for i, item in enumerate(val):
                if isinstance(item, str):
                    data[key][i] = convert_str(item)
                elif isinstance(item, dict):
                    for k, v in item.items():
                        if isinstance(v, str):
                            data[key][i][k] = convert_str(v)      
    return data

In [16]:
luke = to_numeric(luke)

In [17]:
pprint(luke)

{'birth_year': '19BBY',
 'created': '2014-12-09T13:50:51.644000Z',
 'edited': '2014-12-20T21:17:56.891000Z',
 'eye_color': 'blue',
 'films': [{'characters': ['https://swapi.co/api/people/1/',
                           'https://swapi.co/api/people/2/',
                           'https://swapi.co/api/people/3/',
                           'https://swapi.co/api/people/4/',
                           'https://swapi.co/api/people/5/',
                           'https://swapi.co/api/people/10/',
                           'https://swapi.co/api/people/13/',
                           'https://swapi.co/api/people/14/',
                           'https://swapi.co/api/people/18/',
                           'https://swapi.co/api/people/20/',
                           'https://swapi.co/api/people/21/',
                           'https://swapi.co/api/people/22/',
                           'https://swapi.co/api/people/23/',
                           'https://swapi.co/api/people/24/',
      

                             'https://swapi.co/api/people/7/',
                             'https://swapi.co/api/people/8/',
                             'https://swapi.co/api/people/9/',
                             'https://swapi.co/api/people/11/',
                             'https://swapi.co/api/people/43/',
                             'https://swapi.co/api/people/62/'],
               'rotation_period': 23,
               'surface_water': 1,
               'terrain': 'desert',
               'url': 'https://swapi.co/api/planets/1/'},
 'mass': 77,
 'name': 'Luke Skywalker',
 'skin_color': 'fair',
 'species': [{'average_height': 180,
              'average_lifespan': 120,
              'classification': 'mammal',
              'created': '2014-12-10T13:52:11.567000Z',
              'designation': 'sentient',
              'edited': '2015-04-17T06:59:55.850671Z',
              'eye_colors': 'brown, blue, green, hazel, grey, amber',
              'films': ['https://swapi.co/api/fi

## Insertion

### Single inserts

In [18]:
result = people.insert_one(data)

In [19]:
db.list_collection_names()

['people']

### Bulk inserts

In [20]:
xs = [to_numeric(convert(requests.get('https://swapi.co/api/people/%i' % i).json())) 
      for i in range(2, 11)]

In [21]:
result = people.insert_many(xs)

In [22]:
result.inserted_ids

[ObjectId('5b8fc6062b84ad7967ef9949'),
 ObjectId('5b8fc6062b84ad7967ef994a'),
 ObjectId('5b8fc6062b84ad7967ef994b'),
 ObjectId('5b8fc6062b84ad7967ef994c'),
 ObjectId('5b8fc6062b84ad7967ef994d'),
 ObjectId('5b8fc6062b84ad7967ef994e'),
 ObjectId('5b8fc6062b84ad7967ef994f'),
 ObjectId('5b8fc6062b84ad7967ef9950'),
 ObjectId('5b8fc6062b84ad7967ef9951')]

## Queries

In [23]:
people.find_one(
    # search criteria
    {'name': 'Luke Skywalker'}, 
    # values to return
    {'name': True, 
     'hair_color': True,
     'skin_color': True, 
     'eye_color': True
    } 
)

{'_id': ObjectId('5b8fc5ee2b84ad7967ef9948'),
 'eye_color': 'blue',
 'hair_color': 'blond',
 'name': 'Luke Skywalker',
 'skin_color': 'fair'}

### Using object ID

Note that ObjectID is NOT a string. You must convert a string to ObjectID before use.

In [24]:
result.inserted_ids[0]

ObjectId('5b8fc6062b84ad7967ef9949')

In [25]:
people.find_one(
    result.inserted_ids[0],
    {'name': True, 'hair_color': True, 'skin_color': True, 'eye_color': True}
)

{'_id': ObjectId('5b8fc6062b84ad7967ef9949'),
 'eye_color': 'yellow',
 'hair_color': 'n/a',
 'name': 'C-3PO',
 'skin_color': 'gold'}

### Bulk queries

In [26]:
for person in people.find({'gender': 'male'}):
    print(person['name'])

Luke Skywalker
Darth Vader
Owen Lars
Biggs Darklighter
Obi-Wan Kenobi


In [27]:
for x in people.find(
    {'gender': 'male'},             
    {
        'name': True,
        'gender': True
    }
): 
    print(x)

{'name': 'Luke Skywalker', 'gender': 'male', '_id': ObjectId('5b8fc5ee2b84ad7967ef9948')}
{'name': 'Darth Vader', 'gender': 'male', '_id': ObjectId('5b8fc6062b84ad7967ef994b')}
{'name': 'Owen Lars', 'gender': 'male', '_id': ObjectId('5b8fc6062b84ad7967ef994d')}
{'name': 'Biggs Darklighter', 'gender': 'male', '_id': ObjectId('5b8fc6062b84ad7967ef9950')}
{'name': 'Obi-Wan Kenobi', 'gender': 'male', '_id': ObjectId('5b8fc6062b84ad7967ef9951')}


#### Using regex search

In [28]:
for x in people.find(
    {
        'name': {'$regex': '^L'},
    },
    {
        'name': True, 
        'gender': True, 
        '_id': False
    }
):
    pprint(x)

{'gender': 'male', 'name': 'Luke Skywalker'}
{'gender': 'female', 'name': 'Leia Organa'}


Alternative using Python regular expressions.

In [29]:
import re

name_pat = re.compile(r'^l', re.IGNORECASE)

In [30]:
for x in people.find(
    {
        'name': name_pat,
    },
    {
        'name': True,
        'gender': True,
        '_id': False
    }
):
    pprint(x)

{'gender': 'male', 'name': 'Luke Skywalker'}
{'gender': 'female', 'name': 'Leia Organa'}


#### Using relational operators

In [31]:
for x in people.find(
    {
        'mass': {'$gt': 100},
    },
    {
        'name': True, 
        'mass': True, 
        '_id': False
    }
):
    pprint(x)

{'mass': 136, 'name': 'Darth Vader'}
{'mass': 120, 'name': 'Owen Lars'}


In [32]:
mass_range = {'$lt': 100, '$gt': 50}

In [34]:
for x in people.find(
    {
        'mass': mass_range,
    },
    {
        'name': True, 
        'mass': True, 
        '_id': False
    }
):
    print(x)

{'mass': 77, 'name': 'Luke Skywalker'}
{'mass': 75, 'name': 'C-3PO'}
{'mass': 75, 'name': 'Beru Whitesun lars'}
{'mass': 84, 'name': 'Biggs Darklighter'}
{'mass': 77, 'name': 'Obi-Wan Kenobi'}


In [35]:
for x in people.find(
    {
        'mass': mass_range,
    },
    {
        'name': True, 
        'mass': True,
        '_id': False
    }
):
    pprint(x)

{'mass': 77, 'name': 'Luke Skywalker'}
{'mass': 75, 'name': 'C-3PO'}
{'mass': 75, 'name': 'Beru Whitesun lars'}
{'mass': 84, 'name': 'Biggs Darklighter'}
{'mass': 77, 'name': 'Obi-Wan Kenobi'}


#### Nested search

In [36]:
for x in people.find(
    {
        'species.name': 'Human',
    },
    {
        'name': True, 
        'species.name': True, 
        '_id': False
    }
):
    pprint(x)

{'name': 'Luke Skywalker', 'species': [{'name': 'Human'}]}
{'name': 'Darth Vader', 'species': [{'name': 'Human'}]}
{'name': 'Leia Organa', 'species': [{'name': 'Human'}]}
{'name': 'Owen Lars', 'species': [{'name': 'Human'}]}
{'name': 'Beru Whitesun lars', 'species': [{'name': 'Human'}]}
{'name': 'Biggs Darklighter', 'species': [{'name': 'Human'}]}
{'name': 'Obi-Wan Kenobi', 'species': [{'name': 'Human'}]}


#### Matching multiple criteria

In [37]:
for x in people.find(
    {
        'starships.cost_in_credits': {'$lt': 250000},
        'starships.max_atmosphering_speed': {'$gt': 500},
        'starships.passengers': {'$gt': 0}
    },
    {
        'name': True, 
        'starship.name': True, 
        'starships.max_atmosphering_speed': True,
        'starships.passengers': True,
        'starships.cost_in_credits': True,     
        '_id': False
    }
):
    pprint(x)

{'name': 'Luke Skywalker',
 'starships': [{'cost_in_credits': 149999,
                'max_atmosphering_speed': 1050,
                'passengers': 0},
               {'cost_in_credits': 240000,
                'max_atmosphering_speed': 850,
                'passengers': 20}]}
{'name': 'Obi-Wan Kenobi',
 'starships': [{'cost_in_credits': 180000,
                'max_atmosphering_speed': 1150,
                'passengers': 0},
               {'cost_in_credits': 125000000,
                'max_atmosphering_speed': 1050,
                'passengers': 48247},
               {'cost_in_credits': 'unknown',
                'max_atmosphering_speed': 1050,
                'passengers': 3},
               {'cost_in_credits': 320000,
                'max_atmosphering_speed': 1500,
                'passengers': 0},
               {'cost_in_credits': 168000,
                'max_atmosphering_speed': 1100,
                'passengers': 0}]}


#### Matching multiple criteria simultaneously

In [38]:
for x in people.find(
    {
        'starships': {
            '$elemMatch': { 
                'cost_in_credits': {'$lt': 250000},
                'max_atmosphering_speed': {'$gt': 500},
                'passengers': {'$gt': 1}
            }
        }
    },
    {
        'name': True, 
        'starship.name': True, 
        'starships.max_atmosphering_speed': True,
        'starships.passengers': True,
        'starships.cost_in_credits': True,     
        '_id': False
    }
):
    pprint(x)

{'name': 'Luke Skywalker',
 'starships': [{'cost_in_credits': 149999,
                'max_atmosphering_speed': 1050,
                'passengers': 0},
               {'cost_in_credits': 240000,
                'max_atmosphering_speed': 850,
                'passengers': 20}]}


## Indexes

In [39]:
people.find({}).explain

<bound method Cursor.explain of <pymongo.cursor.Cursor object at 0x104c2c9e8>>

In [40]:
people.find({'name': 'Luke Skywalker'}).explain()

{'executionStats': {'allPlansExecution': [],
  'executionStages': {'advanced': 1,
   'direction': 'forward',
   'docsExamined': 10,
   'executionTimeMillisEstimate': 0,
   'filter': {'name': {'$eq': 'Luke Skywalker'}},
   'invalidates': 0,
   'isEOF': 1,
   'nReturned': 1,
   'needTime': 10,
   'needYield': 0,
   'restoreState': 0,
   'saveState': 0,
   'stage': 'COLLSCAN',
   'works': 12},
  'executionSuccess': True,
  'executionTimeMillis': 0,
  'nReturned': 1,
  'totalDocsExamined': 10,
  'totalKeysExamined': 0},
 'ok': 1.0,
 'queryPlanner': {'indexFilterSet': False,
  'namespace': 'starwars.people',
  'parsedQuery': {'name': {'$eq': 'Luke Skywalker'}},
  'plannerVersion': 1,
  'rejectedPlans': [],
  'winningPlan': {'direction': 'forward',
   'filter': {'name': {'$eq': 'Luke Skywalker'}},
   'stage': 'COLLSCAN'}},
 'serverInfo': {'gitVersion': 'fc1573ba18aee42f97a3bb13b67af7d837826b47',
  'host': 'eris',
  'port': 27017,
  'version': '4.0.2'}}

In [41]:
people.create_index('name')

'name_1'

In [42]:
people.find({'name': 'Luke Skywalker'}).explain()

{'executionStats': {'allPlansExecution': [],
  'executionStages': {'advanced': 1,
   'alreadyHasObj': 0,
   'docsExamined': 1,
   'executionTimeMillisEstimate': 0,
   'inputStage': {'advanced': 1,
    'direction': 'forward',
    'dupsDropped': 0,
    'dupsTested': 0,
    'executionTimeMillisEstimate': 0,
    'indexBounds': {'name': ['["Luke Skywalker", "Luke Skywalker"]']},
    'indexName': 'name_1',
    'indexVersion': 2,
    'invalidates': 0,
    'isEOF': 1,
    'isMultiKey': False,
    'isPartial': False,
    'isSparse': False,
    'isUnique': False,
    'keyPattern': {'name': 1},
    'keysExamined': 1,
    'multiKeyPaths': {'name': []},
    'nReturned': 1,
    'needTime': 0,
    'needYield': 0,
    'restoreState': 0,
    'saveState': 0,
    'seeks': 1,
    'seenInvalidated': 0,
    'stage': 'IXSCAN',
    'works': 2},
   'invalidates': 0,
   'isEOF': 1,
   'nReturned': 1,
   'needTime': 0,
   'needYield': 0,
   'restoreState': 0,
   'saveState': 0,
   'stage': 'FETCH',
   'works': 2

## Aggregate Queries

In [43]:
people.count_documents({'species.name': 'Human'})

7

### Using aggregate

The `aggregate` function runs a pipeline of commands, and uses the `$group` operator to summarize results.

In [44]:
from collections import OrderedDict

Filter and count

In [45]:
cmds = [
     {'$match': {'species.name': 'Human'}},
     {'$group': {'_id': '$species.name', 'count': {'$sum': 1}}},
]

In [46]:
pprint(list(people.aggregate(cmds)))

[{'_id': ['Human'], 'count': 7}]


Filter and find total mass

In [47]:
cmds = [
     {'$match': {'species.name': 'Human'}},
     {'$group': {'_id': '$species.name', 'total_mass': {'$sum': '$mass'}}},
]

In [48]:
pprint(list(people.aggregate(cmds)))

[{'_id': ['Human'], 'total_mass': 618}]


Total mass of each species

In [49]:
cmds = [
     {'$group': {'_id': '$species.name', 'total_mass': {'$sum': '$mass'}}},
]

In [50]:
pprint(list(people.aggregate(cmds)))

[{'_id': ['Droid'], 'total_mass': 139}, {'_id': ['Human'], 'total_mass': 618}]


Total and average mass of each species

In [51]:
cmds = [
     {
         '$group': {
             '_id': '$species.name',
             'total_mass': {'$sum': '$mass'},
             'avg_mass': {'$avg': '$mass'}
         }
     },
]

In [52]:
pprint(list(people.aggregate(cmds)))

[{'_id': ['Droid'], 'avg_mass': 46.333333333333336, 'total_mass': 139},
 {'_id': ['Human'], 'avg_mass': 88.28571428571429, 'total_mass': 618}]


### Using MapReduce

With `MapReduce` you get the full power of JavaScript, but it is more complex and often less efficient. You should use `aggregate` in preference to `map_reduce` in most cases.

- In the map stage, you create a (key, value) pair
- In the reduce stage, you perform a reduction (e.g. sum) of the values associated with each key

In [53]:
from bson.code import Code

Count the number by eye_color.

In [54]:
mapper = Code('''
function() {
    emit(this.eye_color, 1);
}
''')

reducer = Code('''
function (key, values) {
    var total = 0;
    for (var i = 0; i < values.length; i++) {
        total += values[i];
    }
    return total;
}
''')

result = people.map_reduce(
    mapper, 
    reducer, 
    'result1'
)

In [55]:
for doc in result.find():
    pprint(doc)

{'_id': 'blue', 'value': 3.0}
{'_id': 'blue-gray', 'value': 1.0}
{'_id': 'brown', 'value': 2.0}
{'_id': 'red', 'value': 2.0}
{'_id': 'yellow', 'value': 2.0}


The output is also stored in the `result1` collection we specified.

In [56]:
list(db.result1.find())

[{'_id': 'blue', 'value': 3.0},
 {'_id': 'blue-gray', 'value': 1.0},
 {'_id': 'brown', 'value': 2.0},
 {'_id': 'red', 'value': 2.0},
 {'_id': 'yellow', 'value': 2.0}]

Using JavaScript Array functions to simplify code.

In [57]:
mapper = Code('''
function() {
    emit(this.eye_color, 1);
}
''')

reducer = Code('''
function (key, values) {
    return Array.sum(values);
}
''')

result = people.map_reduce(
    mapper, 
    reducer, 
    'result2'
)

In [58]:
for doc in result.find():
    pprint(doc)

{'_id': 'blue', 'value': 3.0}
{'_id': 'blue-gray', 'value': 1.0}
{'_id': 'brown', 'value': 2.0}
{'_id': 'red', 'value': 2.0}
{'_id': 'yellow', 'value': 2.0}


Find avergae mass by gender.

In [59]:
mapper = Code('''
function() {
    emit(this.gender, this.mass);
}
''')

reducer = Code('''
function (key, values) {
    return Array.avg(values);
}
''')

result = people.map_reduce(
    mapper, 
    reducer, 
    'result3'
)

In [60]:
for doc in result.find():
    pprint(doc)

{'_id': 'female', 'value': 62.0}
{'_id': 'male', 'value': 98.8}
{'_id': 'n/a', 'value': 46.333333333333336}


Count number of members in each species

In [61]:
mapper = Code('''
function() {
    this.species.map(function(z) {
      emit(z.name, 1);
    })
}
''')

reducer = Code('''
function (key, values) {
    return Array.sum(values);
}
''')

result = people.map_reduce(
    mapper, 
    reducer, 
    'result3'
)

In [62]:
for doc in result.find():
    pprint(doc)

{'_id': 'Droid', 'value': 3.0}
{'_id': 'Human', 'value': 7.0}


## Geospatial queries

In [63]:
crime = db.crime

In [64]:
import json

In [65]:
path = 'data/crime-mapping.geojson'

with open(path) as f:
    datastore = json.load(f)

In [66]:
results = crime.insert_many(datastore['features'])

In [67]:
crime.find_one({})

{'_id': ObjectId('5b8fc6472b84ad7967ef9952'),
 'geometry': {'coordinates': [-78.78200313, 35.760212065], 'type': 'Point'},
 'properties': {'activity_date': None,
  'apartment_complex': None,
  'beat_number': '112',
  'chrgcnt': None,
  'crime_category': 'ALL OTHER',
  'crime_type': 'ALL OTHER - ESCAPE FROM CUSTODY OR RESIST ARREST',
  'crimeday': 'THURSDAY',
  'date_from': '2017-11-30',
  'date_to': '11/30/2017',
  'district': 'D3',
  'domestic': 'N',
  'incident_number': '17010528',
  'lat': 35.760212065,
  'location_category': 'TOWN OWNED',
  'lon': -78.78200313,
  'map_reference': 'P027',
  'offensecategory': 'All Other Offenses',
  'period': ['Everything', 'Last Year'],
  'phxcommunity': 'No',
  'phxrecordstatus': None,
  'phxstatus': None,
  'radio': 'Everything,Last Year',
  'record': 3145,
  'residential_subdivision': 'SHOPPES OF KILDAIRE',
  'street': 'KILDAIRE FARM RD',
  'time_from': -62135553600,
  'time_to': -62135553600,
  'timeframe': ['Last Year'],
  'ucr': '2650',
  'vi

In [68]:
crime.find_one({},
              {
                  'geometry': 1,
                  '_id': 0,
              }
              )

{'geometry': {'coordinates': [-78.78200313, 35.760212065], 'type': 'Point'}}

In [69]:
crime.create_index([('geometry', GEOSPHERE)])

'geometry_2dsphere'

List 5 crimes near the location

In [70]:
loc = SON([('type', 'Point'), ('coordinates', [-78.78200313, 35.760212065])])

for doc in crime.find(
    {
        'geometry' : SON([('$near', {'$geometry' : loc})])
    },
    {
        '_id': 0,
        'properties.crime_type': 1,
        'properties.date_from': 1
    }
).limit(5):
    pprint(doc)

{'properties': {'crime_type': 'ALL OTHER - ESCAPE FROM CUSTODY OR RESIST '
                              'ARREST',
                'date_from': '2017-11-30'}}
{'properties': {'crime_type': 'LARCENY - AUTO PARTS OR ACCESSORIES',
                'date_from': '2018-03-20'}}
{'properties': {'crime_type': 'COUNTERFEITING - USING',
                'date_from': '2018-08-05'}}
{'properties': {'crime_type': 'DRUGS - DRUG VIOLATIONS '
                              '(POSS./SELL/MAN./DEL./TRNSPRT/CULT.)',
                'date_from': '2017-11-30'}}
{'properties': {'crime_type': 'VANDALISM - DAMAGE TO PROPERTY',
                'date_from': '2018-03-26'}}


List crimes committed nearby (within 200 m)

In [71]:
loc = SON([('type', 'Point'), ('coordinates', [-78.78200313, 35.760212065])])

for doc in crime.find(
    {
        'geometry' : SON([('$geoNear', {'$geometry' : loc, '$minDistance': 1e-6, '$maxDistance': 200})]),
    },
    {
        '_id': 0,
        'geometry.coordinates': 1,
        'properties.crime_type': 1,
        'properties.date_from': 1
    }
):
    pprint(doc)

{'geometry': {'coordinates': [-78.78102423, 35.7607323]},
 'properties': {'crime_type': 'ASSAULT - SIMPLE - ALL OTHER',
                'date_from': '2018-02-14'}}
{'geometry': {'coordinates': [-78.78131931, 35.761138061]},
 'properties': {'crime_type': 'VANDALISM - GRAFFITI',
                'date_from': '2018-07-20'}}
{'geometry': {'coordinates': [-78.7827814, 35.759087052]},
 'properties': {'crime_type': 'VANDALISM - GRAFFITI',
                'date_from': '2018-07-29'}}
